# Title: Pokemon classification on iOS using Convolutional Neural Network based on Keras

---




***First you need to download dataset and Swift code to deploy image classification on iphone from Google Drive, since the files are too large to upload on github.***
Please follow the instructions:
1. Click the links to download [dataset]() and [pokemon_iOS]() from google drive.
2. Click `dataset` / `pokemon_iOS` icon on the top with a down-arrow to download the whole folder. 
3. Click `Download` and wait for downloading.
4. (If needed) Unzipping the file.
4. Move the `dataset` and `pokemon_iOS` folder to the folder containing this `jupyter notebook`. In this study, the folder is `Project`.

In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.models import load_model
import coremltools
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split



import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os


def build_SmallerVGGNet(width, height, depth, classes):
    # initialize the model along with the input shape to be
    # "channels last" and the channels dimension itself
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    # if we are using "channels first", update the input shape
    # and channels dimension
    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
        chanDim = 1

    # CONV => RELU => POOL
    model.add(Conv2D(32, (3, 3), padding="same",
        input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    # return the constructed network architecture
    return model

Duplicate key in file '/Users/raymond/.matplotlib/matplotlibrc', line 2 ('backend: TkAgg')
Duplicate key in file '/Users/raymond/.matplotlib/matplotlibrc', line 3 ('backend: TkAgg')


In [14]:


# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 50
INIT_LR = 1e-3
BS = 64
IMAGE_DIMS = (96, 96, 3)

plotting = 'plot.png'
save_path = 'Pokemon.model'

# initialize the data and labels
data = []
labels = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images("./dataset")))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
    data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for validation and testing
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.2, random_state=42)
(valX, testX, valY, testY) = train_test_split(testX,
    testY, test_size=0.2, random_state=42)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

# initialize the model
print("[INFO] compiling model...")
model = build_SmallerVGGNet(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
    depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training network...")
stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
checkpoint = ModelCheckpoint(save_path, monitor='val_acc', mode='max', verbose=1, save_best_only=True)

H = model.fit(
    x=aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, 
    verbose=1,
    callbacks = [checkpoint]
)

# save the model to disk
print("[INFO] serializing network...")
#model.save(save_path, save_format="h5")

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open('label.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(plotting)

[INFO] loading images...
[INFO] data matrix: 252.07MB
[INFO] compiling model...
[INFO] training network...
Epoch 1/50
14/14 [==============================] - 19s 1s/step - loss: 1.9067 - accuracy: 0.4969 - val_loss: 1.4968 - val_accuracy: 0.3404
Epoch 2/50
14/14 [==============================] - 17s 1s/step - loss: 0.6600 - accuracy: 0.8102 - val_loss: 1.6524 - val_accuracy: 0.3191
Epoch 3/50
14/14 [==============================] - 17s 1s/step - loss: 0.5028 - accuracy: 0.8369 - val_loss: 1.9118 - val_accuracy: 0.2340
Epoch 4/50
14/14 [==============================] - 17s 1s/step - loss: 0.4126 - accuracy: 0.8601 - val_loss: 1.8732 - val_accuracy: 0.2340
Epoch 5/50
14/14 [==============================] - 18s 1s/step - loss: 0.4218 - accuracy: 0.8702 - val_loss: 1.5960 - val_accuracy: 0.3830
Epoch 6/50
14/14 [==============================] - 18s 1s/step - loss: 0.3237 - accuracy: 0.9044 - val_loss: 1.9731 - val_accuracy: 0.3191
Epoch 7/50
14/14 [==============================] - 1

Epoch 38/50
14/14 [==============================] - 17s 1s/step - loss: 0.1305 - accuracy: 0.9476 - val_loss: 0.9596 - val_accuracy: 0.8511
Epoch 39/50
14/14 [==============================] - 17s 1s/step - loss: 0.1228 - accuracy: 0.9590 - val_loss: 1.3560 - val_accuracy: 0.8298
Epoch 40/50
14/14 [==============================] - 17s 1s/step - loss: 0.0734 - accuracy: 0.9753 - val_loss: 0.4710 - val_accuracy: 0.8298
Epoch 41/50
14/14 [==============================] - 18s 1s/step - loss: 0.1328 - accuracy: 0.9501 - val_loss: 1.2117 - val_accuracy: 0.8511
Epoch 42/50
14/14 [==============================] - 17s 1s/step - loss: 0.1467 - accuracy: 0.9466 - val_loss: 1.1571 - val_accuracy: 0.8511
Epoch 43/50
14/14 [==============================] - 18s 1s/step - loss: 0.0880 - accuracy: 0.9667 - val_loss: 1.2905 - val_accuracy: 0.8298
Epoch 44/50
14/14 [==============================] - 17s 1s/step - loss: 0.0794 - accuracy: 0.9692 - val_loss: 0.7946 - val_accuracy: 0.8298
Epoch 45/50
1

In [12]:
pretrain = ResNet50(weights = 'imagenet',
              include_top = False,
              input_shape = IMAGE_DIMS)

 
# Model definition
# Create the model
model = Sequential()

# Freeze the convolutional base of VGG16 to prevent the pre-trained weights being updated 
# during training inorder to extract features
pretrain.trainable=False
 
# Add the vgg convolutional base model
model.add(pretrain)
 
# Add new layers
model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())                 # normalize and scale inputs or activations
model.add(Dropout(0.2))                         # applies dopout to the input which will randomly disable 20% of hidden units

# add densely-connected NN layer with 128 hidden units
model.add(Dense(units=128, activation='relu')) # use ReLU activation function
model.add(BatchNormalization())                # normalize and scale inputs or activations
model.add(Dropout(0.2))     

model.add(Dense(5, activation='softmax'))

model.summary()


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS),
              metrics=['acc'])

# Train the model
stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
checkpoint = ModelCheckpoint('Res50.hdf5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

historyres = model.fit(
    x=aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, 
    verbose=1,
    callbacks = [checkpoint]
)


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 3, 3, 2048)        23587712  
_________________________________________________________________
flatten_4 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              18875392  
_________________________________________________________________
batch_normalization_20 (Batc (None, 1024)              4096      
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               131200    
_________________________________________________________________
batch_normalization_21 (Batc (None, 128)              

In [13]:
pretrain = InceptionV3(weights = 'imagenet',
              include_top = False,
              input_shape = IMAGE_DIMS)

 
# Model definition
# Create the model
model = Sequential()

# Freeze the convolutional base of VGG16 to prevent the pre-trained weights being updated 
# during training inorder to extract features
pretrain.trainable=False
 
# Add the vgg convolutional base model
model.add(pretrain)
 
# Add new layers
model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())                 # normalize and scale inputs or activations
model.add(Dropout(0.2))                         # applies dopout to the input which will randomly disable 20% of hidden units

# add densely-connected NN layer with 128 hidden units
model.add(Dense(units=128, activation='relu')) # use ReLU activation function
model.add(BatchNormalization())                # normalize and scale inputs or activations
model.add(Dropout(0.2))     

model.add(Dense(5, activation='softmax'))

model.summary()


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS),
              metrics=['acc'])

# Train the model
stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
checkpoint = ModelCheckpoint('Incep.model', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

historyin = model.fit(
    x=aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, 
    verbose=1,
    callbacks = [checkpoint]
)



87916544/87910968 [==============================] - 96s 1us/step
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 1, 1, 2048)        21802784  
_________________________________________________________________
flatten_5 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_116 (Bat (None, 1024)              4096      
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               131200    
______________________________________________________


Epoch 00033: val_acc did not improve from 0.78723
Epoch 34/50
14/14 [==============================] - 13s 880ms/step - loss: 0.3906 - acc: 0.8439 - val_loss: 0.6076 - val_acc: 0.7872

Epoch 00034: val_acc did not improve from 0.78723
Epoch 35/50
14/14 [==============================] - 12s 815ms/step - loss: 0.3960 - acc: 0.8549 - val_loss: 0.5726 - val_acc: 0.7872

Epoch 00035: val_acc did not improve from 0.78723
Epoch 36/50
14/14 [==============================] - 9s 597ms/step - loss: 0.4003 - acc: 0.8569 - val_loss: 0.6369 - val_acc: 0.7234

Epoch 00036: val_acc did not improve from 0.78723
Epoch 37/50
14/14 [==============================] - 10s 666ms/step - loss: 0.4097 - acc: 0.8440 - val_loss: 0.6503 - val_acc: 0.7447

Epoch 00037: val_acc did not improve from 0.78723
Epoch 38/50
14/14 [==============================] - 11s 752ms/step - loss: 0.3860 - acc: 0.8470 - val_loss: 0.5960 - val_acc: 0.8085

Epoch 00038: val_acc improved from 0.78723 to 0.80851, saving model to Ince

In [2]:


# converting to CoreML

# load the class labels
print("[INFO] loading class labels from label binarizer")
lb = pickle.loads(open('label.pickle', "rb").read())
class_labels = lb.classes_.tolist()
print("[INFO] class labels: {}".format(class_labels))

# load the trained convolutional neural network
print("[INFO] loading model...")
model = load_model('Pokemon.model')

# convert the model to coreml format
print("[INFO] converting model")
coreml_model = coremltools.converters.convert(model,
    input_names="image",
    image_input_names="image",
    image_scale=1/255.0,
    class_labels=class_labels,
    is_bgr=True)

# save the model to disk
output = "pokemon.mlmodel"
print("[INFO] saving model as {}".format(output))
coreml_model.save(output)


[INFO] loading class labels from label binarizer
[INFO] class labels: ['bulbasaur', 'charmander', 'mewtwo', 'pikachu', 'squirtle']
[INFO] loading model...


/Users/raymond/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


[INFO] converting model


Translating MIL ==> MLModel Ops: 100%|██████████| 101/101 [00:01<00:00, 77.93 ops/s]


[INFO] saving model as pokemon.mlmodel
